# ⚡ Multi-Aspect Solutions: The Parallelization Pattern

Welcome to parallel processing! This pattern shines when:
- Tasks can be divided into independent subtasks
- Multiple perspectives improve the solution
- Each aspect needs focused attention

Perfect for generating different OpenSearch solution approaches! 🚀

In [ ]:
import boto3

from utils.retrieval_utils import get_chroma_os_docs_collection, ChromaDBRetrievalClient

# Initialize the Bedrock client
REGION = 'us-west-2'
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name=REGION)

# We've pushed the retrieval client from the prompt chaining notebook to the retrieval utils for simplicity
chroma_os_docs_collection: ChromaDBRetrievalClient = get_chroma_os_docs_collection()

print("✅ Client setup and retrieval client complete!")

# Helpers
Import the same helpers for bedrock

In [ ]:
from typing import Type, Dict, Any, List

# We pushed the base propmt from the previous lab to a a base prompt file.
from utils.base_prompt import BasePrompt
from utils.retrieval_utils import RetrievalResult

def call_bedrock(prompt: BasePrompt) -> str:
    kwargs = {
        "modelId": 'us.amazon.nova-micro-v1:0', # Lets use nova micro because it's really fast
        "inferenceConfig": prompt.hyperparams,
        "messages": prompt.to_bedrock_messages(),
        "system": prompt.to_bedrock_system(),
    }

    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return converse_response['output']['message']['content'][0]['text']

## 1. Creating Our Parallel Solution Generator

We'll generate three different solutions simultaneously:
1. Basic solution for beginners
2. Advanced solution for experts
3. Cost-optimized solution

First lets create our prompt

In [ ]:
from typing import TypedDict, Dict, Any, List, Annotated
import operator
from langgraph.graph import StateGraph, START, END

# Define the system prompt
SOLUTION_SYSTEM_PROMPT = """
You are a helpful assistant specializing in OpenSearch documentation and support.
"""

# Define reusable prompt templates as constants
BEGINNER_PROMPT_TEMPLATE = """
Create a beginner-friendly solution for this OpenSearch question:
{question}

Focus on:
- Simple, step-by-step instructions
- Basic concepts and terminology
- Common pitfalls to avoid
- Default configurations
"""

EXPERT_PROMPT_TEMPLATE = """
Create an advanced, expert-level solution for this OpenSearch question:
{question}

Include:
- Advanced configurations
- Performance optimizations
- Best practices
- Edge cases and considerations
"""

COST_PROMPT_TEMPLATE = """
Create a cost-optimized solution for this OpenSearch question:
{question}

Focus on:
- Resource efficiency
- Infrastructure costs
- Performance/cost tradeoffs
- Cost monitoring and optimization
"""

# Define prompt classes that inherit from BasePrompt
class BeginnerPrompt(BasePrompt):
    system_prompt: str = SOLUTION_SYSTEM_PROMPT
    user_prompt: str = BEGINNER_PROMPT_TEMPLATE

class ExpertPrompt(BasePrompt):
    system_prompt: str = SOLUTION_SYSTEM_PROMPT
    user_prompt: str = EXPERT_PROMPT_TEMPLATE

class CostPrompt(BasePrompt):
    system_prompt: str = SOLUTION_SYSTEM_PROMPT
    user_prompt: str = COST_PROMPT_TEMPLATE

# Define the WorkflowState using TypedDict.
class WorkflowState(TypedDict):
    question: str
    beginner_solution: str
    expert_solution: str  
    cost_solution: str
    final_output: str


Next lets define our nodes

In [ ]:
def parallel_start(state: WorkflowState) -> WorkflowState:
    """Starts the parallel workflow"""
    return state

def generate_beginner_solution(state: WorkflowState) -> Dict:
    """Generates a beginner-friendly solution"""
    beginner_prompt: BasePrompt = BeginnerPrompt(inputs={"question": state["question"]})
    solution: str = call_bedrock(beginner_prompt)
    # Only return the field this function is responsible for
    return {"beginner_solution": solution}

def generate_expert_solution(state: WorkflowState) -> Dict:
    """Generates an expert-level solution"""
    expert_prompt: BasePrompt = ExpertPrompt(inputs={"question": state["question"]})
    solution: str = call_bedrock(expert_prompt)
    # Only return the field this function is responsible for
    return {"expert_solution": solution}

def generate_cost_solution(state: WorkflowState) -> Dict:
    """Generates a cost-optimized solution"""
    cost_prompt: BasePrompt = CostPrompt(inputs={"question": state["question"]})
    solution: str = call_bedrock(cost_prompt)
    # Only return the field this function is responsible for
    return {"cost_solution": solution}

def format_output(state: WorkflowState) -> WorkflowState:
    """Formats the parallel solutions into a clear response"""
    state["final_output"] = f"""
    # OpenSearch Solution Approaches
    
    ## 📘 Beginner-Friendly Solution
    {state["beginner_solution"]}
    
    ## 🎯 Expert-Level Solution
    {state["expert_solution"]}
    
    ## 💰 Cost-Optimized Solution
    {state["cost_solution"]}
    """
    return state

def init_state(question: str) -> WorkflowState:
    """Initialize the workflow state with a question."""
    return WorkflowState(
        question=question,
        beginner_solution="",
        expert_solution="",
        cost_solution="",
        final_output=""
    )

And lastly lets compile our graph

In [ ]:
def create_parallel_workflow() -> StateGraph:
    """Creates a workflow for generating parallel solutions"""
    workflow = StateGraph(WorkflowState)
    
    # Add nodes to our graph
    workflow.add_node("parallelizer", parallel_start)
    workflow.add_node("beginner", generate_beginner_solution)
    workflow.add_node("expert", generate_expert_solution)
    workflow.add_node("cost", generate_cost_solution)
    workflow.add_node("format", format_output)
    
    # Create the parallel workflow
    # From START, branch to all three solution generators
    workflow.add_edge(START, "parallelizer")

    # Each parallel node leads to the parallelizer
    workflow.add_edge("parallelizer", "beginner")
    workflow.add_edge("parallelizer", "expert")
    workflow.add_edge("parallelizer", "cost")
    
    # Each solution node leads to format when all are complete.
    workflow.add_edge(["beginner", "expert", "cost"], "format")
    
    # Format leads to END
    workflow.add_edge("format", END)
    
    # Compile and return the workflow
    return workflow.compile()


graph: StateGraph = create_parallel_workflow()


In [ ]:
question = "How to scale OpenSearch clusters effectively?"
state: WorkflowState = init_state(question)
result: WorkflowState = graph.invoke(state)

print("⚡ Multiple Solution Approaches Generated")
print(result['final_output'])


## 3. Benefits of the Parallelization Pattern

Our parallel solution approach provides several advantages:

✅ Faster overall response time through parallelization

✅ Easy parallelization through LangGraph

Next, we'll explore the orchestrator-workers pattern for complex troubleshooting! 🚀